# 環境を準備しよう
https://github.com/microsoft/semantic-kernel/blob/main/python/samples/learn_resources/your_first_prompt.py


まず、Semantic Kernelと必要なパッケージをインストールしてバージョンを確認ます。

In [1]:
%%capture
%pip install -qU semantic-kernel python-dotenv

In [2]:
from semantic_kernel import __version__

__version__

'1.20.0'

次に、Azure OpenAI Serviceのキー情報を保存している「.env」ファイルを読み込みます。
`.env` ファイルには、次のような設定を記述します。

```bash
SK_AZURE_OPENAI_DEPLOYMENT_NAME=***
SK_AZURE_OPENAI_API_KEY=***
SK_AZURE_OPENAI_ENDPOINT=***
SK_AZURE_OPENAI_API_VERSION=***
```

この情報をコードで読み込みます。

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

# Prompt Tempalteを使って会話する

### Kernel（カーネル）を作成する

Semantic Kernelのメインコンポーネントである Kernel（カーネル）のインスタンスを作成します。

In [4]:
from semantic_kernel import Kernel

kernel = Kernel()

### AIサービス（Azure OpenAI Service）を設定する

Semantic Kernelでは、AIサービスを登録して使用します。<br/>
ここでは、Azure OpenAI Serviceの GPT-4oモデルを利用し、[Chat Completion API](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/how-to/chatgpt)を使います。<br/>
サービスIDは必須ではありませんが、流れを理解しやすくするために設定しておきます。

In [5]:

import os
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

service_id = None
chat_completion_service = AzureChatCompletion(
    deployment_name = os.getenv("SK_AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_key = os.getenv("SK_AZURE_OPENAI_API_KEY"),
    endpoint = os.getenv("SK_AZURE_OPENAI_ENDPOINT"), 
    api_version = os.getenv("SK_AZURE_OPENAI_API_VERSION"), 
    service_id = service_id,
)
kernel.add_service(chat_completion_service)

### チャット履歴を作成する

チャットセッション内でメッセージ履歴を管理するために、チャット履歴オブジェクトを作成します。

In [6]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()

### プロンプトテンプレート作成して登録する

チャット履歴とユーザ入力を設定できるプロンプトテンプレートを作成します。<br/>
テンプレートには過去会話履歴と現在の入力が内容が入れる設定となっていてフォーマットは「Semantic Kernel」形式で作成しました。<br/>
プロンプトテンプレートはプロンプトの設定オブジェクトで設定しておきます。

In [7]:
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

template = """
Previous information from chat:
{{$chat_history}}

User: {{$request}}
Assistant:
"""
prompt_template_config = PromptTemplateConfig(
    template=template,
    name="chat",
    description="Chat with the assistant",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="chat_history", description="The conversation history", is_required=False, default=""),
        InputVariable(name="request", description="The user's request", is_required=True),
    ],
    execution_settings=AzureChatPromptExecutionSettings(),
)

Kernel（カーネル）にプロンプトテンプレートを使って関数として登録します。

In [8]:
chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="ChatBot",
    prompt_template_config=prompt_template_config,
)

### 会話を開始し、AIの応答を受け取る

会話レベルでユーザーの入力を送信し、Chat Completion APIから応答を取得して出力します。<br/>
テンプレートで使いたい変数はKernelArgumentsオブジェクトとしてKernelに渡す必要があります。

In [9]:
from semantic_kernel.functions import KernelArguments
user_input = "Hello, how are you? I'm Jin"
answer = await kernel.invoke(
    function=chat_function,
    arguments=KernelArguments(
        request=user_input,
        chat_history=chat_history,
    ),
)
print("User:", user_input)
chat_history.add_user_message(user_input)
print("Assistant:", answer)
chat_history.add_assistant_message(str(answer))

User: Hello, how are you? I'm Jin
Assistant: Hello Jin! I'm an AI language model, so I don't have feelings, but thanks for asking. How can I assist you today?


次の内容を入力して会話を続けてみます。

In [10]:
user_input = "What is my name?"
answer = await kernel.invoke(
    function=chat_function,
    arguments=KernelArguments(
        request=user_input,
        chat_history=chat_history,
    ),
)
print("User:", user_input)
print("Assistant:", answer)

User: What is my name?
Assistant: Your name is Jin. How can I help you today?
